<img src="logo.png" width="420px">

# <span style="font-family:Cascadia mono; color:#db5a5a">TODOIST AUTOMATION - DIEGO</span>

## <span style="font-family:Cascadia mono; color:#db6a6a">Settings</span>

### Libraries

In [1]:
from todoist_api_python.api import TodoistAPI
from datetime import datetime, timedelta, timezone
import functions as fun
import random
import json
import time

### Runtime

In [2]:
start_time = time.time()

### Dependences

In [3]:
import credentials

### API

In [4]:
api_token = credentials.api_token
api = TodoistAPI(api_token)

## <span style="font-family:Cascadia mono; color:#db6a6a">Data</span>

#### Projects

In [5]:
all_projects = fun.getProjects()
projects_dict_id, projects_dict_name = fun.getProjectsDicts(all_projects)

#### Tasks

In [6]:
def refreshTasks():
    all_tasks = fun.getTasks()
    task_dict_id, task_dict_name = fun.getTasksDicts(all_tasks, projects_dict_id, projects_dict_name)
    return all_tasks, task_dict_id, task_dict_name

In [7]:
all_tasks, task_dict_id, task_dict_name = refreshTasks()

#### Sections

In [8]:
all_sections = fun.getSections()
sections_dict_id, sections_dict_name = fun.getSectionsDicts(all_sections)

#### Test

In [9]:
index = random.randint(0,len(all_tasks))
try:
    print(all_tasks[index]['content'], "-", all_tasks[index]['due']['date'], "-", projects_dict_id[all_tasks[index]['project_id']])
except TypeError:
    print(all_tasks[index]['content'], "-", "No date" , "-",projects_dict_id[all_tasks[index]['project_id']],)

* Practice 1 - No date - Fórmula 1


#### Labels

In [10]:
label_names=[]
for task in all_tasks:
    gross_labels=task['labels']
    for label in gross_labels:
        if label not in label_names:
            label_names.append(label)
label_names

['Med',
 'PC',
 'Recurring',
 'Phone',
 'Short',
 'Home',
 'Long',
 'Outside',
 '❌_streak',
 '✅_streak',
 'Vacations']

#### F1 Calendar

In [11]:
def getF1Calendar():
    return True


## <span style="font-family:Cascadia mono; color:#db6a6a">Orders</span>

#### <span style="font-family:Cascadia mono; color:silver">Create and edit tasks</span>

In [12]:
def createTask(
    content = "Tarea creada automáticamente",
    description = None,
    project_id = projects_dict_name['Inbox'],
    section_id = None,
    parent_id = None,
    order = 1,
    labels = [],
    priority = 1,
    comment_count = 0,
    due_string = None,
    due_date = None,
    due_datetime = None,
    due_lang = None,
    assignee_id = None,
    duration = None,
    duration_unit = None
):
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.add_task(
            assignee_id = assignee_id,
            comment_count = comment_count,
            content = content,
            description = description,
            due_string = due_string,
            due_date = due_date,
            due_datetime = due_datetime,
            due_lang = due_lang,
            duration = duration,
            labels = labels,
            order = order,
            priority = fun.priorityInversal(priority),
            project_id = project_id,
            section_id = section_id,
            parent_id = parent_id,
            duration_unit = duration_unit
        )
        print(f'Task "{task.content}" ({task.id}) was created correctly')
    except Exception as error:
        print(error)
        
def editTask(
    task_id,
    content = None,
    assignee_id = None,
    description = None,
    due_date = None,
    due_string = None,
    due_datetime = None,
    labels = None,
    priority = None,
):
    task = api.get_task(task_id)
    labels = task.labels if labels == None else labels
    content = task.content if content == None else content
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.update_task(
            task_id = task_id,
            assignee_id = assignee_id,
            content = content,
            description = description,
            labels = labels,
            priority = priority,
            due_date = due_date,
            due_string = due_string,
            due_datetime = due_datetime,
        )
        print(f'Task {content} updated correctly to {task["content"]}')
    except Exception as error:
        print(error)

#### <span style="font-family:Cascadia mono; color:silver">Add duration labels</span>

In [13]:
def updateTaskDurationLabel(task_id, task_duration):
    duration_label = fun.getDurationLabel(task_duration)
    task_id = task_id[0]
    task_labels_without_duration = fun.getLabelsWithoutDuration(task_id)
    task_labels_without_duration.append(duration_label)
    editTask(task_id=task_id,
             labels=task_labels_without_duration)

In [14]:
to_update = False
for task in all_tasks:
    if task['duration'] != None and task['due'] != None:
        if fun.getDurationLabel(task['duration']['amount']) not in task['labels']:
            to_update = True
            updateTaskDurationLabel([task['id']], task['duration']['amount'])

if not to_update:
    print('No task to be updated')
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

Task Meeting Goal Spent updated correctly to Meeting Goal Spent
Task Reconocimiento médico updated correctly to Reconocimiento médico


#### <span style="font-family:Cascadia mono; color:silver">Capitalize titles</span>

In [16]:
to_update = False
for task in all_tasks:
    if task["content"][0].upper() != task["content"][0]:
        to_update = True
        task_id = task["id"]
        editTask(task_id = task_id,
            content = task["content"][0].upper() + task["content"][1:])

if not to_update:
    print("No tasks to be updated")
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

Task Montarme yo crm para ver permisos updated correctly to Montarme yo crm para ver permisos
Task Hablar con PsicoLink dinámicas de sesiones individuales y remotas updated correctly to Hablar con PsicoLink dinámicas de sesiones individuales y remotas
Task Picotazo coche updated correctly to Picotazo coche


#### <span style="font-family:Cascadia mono; color:silver">Uncomplete recurring tasks</span>

In [17]:
with open('recurringtasksdiego', 'r') as archivo_json:
    recurringtasks = json.load(archivo_json)

In [18]:
to_update = False
for task_id in recurringtasks.keys():
    if fun.getTask(task_id).is_completed:
        to_update = True
        print(task.id)
        fun.uncompleteTask(task_id)
        editTask(task_id=task_id,
                 due_string="No date")
if not to_update:
    print("No tasks to be updated")
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No tasks to be updated


#### <span style="font-family:Cascadia mono; color:silver">Add labels to birthday tasks</span>

In [19]:
to_update = False
for task in all_tasks:
    if task['project_id']==projects_dict_name['Cumpleaños'] and task['labels']!=['Phone','Short']:
        to_update = True
        editTask(task_id=task['id'],
                 labels=['Phone','Short'])
        
if not to_update:
    print("No tasks to be updated")
else:
    all_tasks, task_dict_id, task_dict_name = refreshTasks()

No tasks to be updated


#### <span style="font-family:Cascadia mono; color:silver">Reminder and completion of habit tracking tasks not being completed</span>

In [20]:
# fun.completeTask('')

#### <span style="font-family:Cascadia mono; color:silver">Fantasy task due</span>

In [21]:
# find next man united premier league game
# take it's day date
# if it's tuesday, wednesday or thursday, move fantasy to tuesday

# editTask('4632052423')

## <span style="font-family:Cascadia mono; color:#db6a6a">Results</span>

### Export all tasks and it's status

In [22]:
tasks = api.get_tasks()
recurringtasks={}
for task in tasks:
    if 'Recurring' in task.labels:
        recurringtasks[task.id]=task.is_completed
with open('recurringtasksdiego', 'w') as file:
    json.dump(recurringtasks, file)

### Runtime

In [23]:
end_time = time.time()

In [24]:
print(f'Runtime: {round(end_time- start_time,3)} seconds')

Runtime: 89.206 seconds
